<a href="https://colab.research.google.com/github/SnigdhaJamwal/Arya_Data_Scientist_Assignment/blob/main/Arya_ai_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import all the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras import layers, models, optimizers, losses, regularizers, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
from google.colab import files 
uploaded = files.upload()


Saving test_set.csv to test_set.csv
Saving training_set.csv to training_set.csv


In [3]:
path = "/content/"

In [4]:
training_data = pd.read_csv(path + "training_set.csv")
test_data = pd.read_csv(path + "test_set.csv")

In [5]:
# get the input and output variables
X = training_data.iloc[:, 1:-1]
y = training_data.iloc[:, -1]
X.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57
count,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,...,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000,3910.000000
mean,0.102990,0.206419,0.284419,0.062074,0.311309,0.095974,0.112320,0.106041,0.091146,0.244345,...,0.033281,0.037493,0.139252,0.015876,0.272971,0.077820,0.043828,5.047150,52.338107,283.059079
std,0.296322,1.253828,0.504352,1.369361,0.656195,0.261455,0.389516,0.398694,0.271417,0.667065,...,0.301611,0.235054,0.276309,0.083600,0.858634,0.256991,0.452862,31.397035,204.445218,578.339858
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.580750,6.000000,35.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066000,0.000000,0.000000,0.000000,0.000000,2.263500,15.000000,94.000000
75%,0.000000,0.000000,0.430000,0.000000,0.387500,0.000000,0.000000,0.000000,0.000000,0.170000,...,0.000000,0.000000,0.188000,0.000000,0.317250,0.054000,0.000000,3.714000,43.000000,264.000000
max,4.340000,14.280000,4.540000,42.810000,9.090000,3.570000,7.270000,11.110000,3.230000,18.180000,...,10.000000,4.385000,9.752000,2.777000,32.478000,6.003000,19.829000,1102.500000,9989.000000,10062.000000


In [6]:
# Checking the missing values in input
X.isnull().sum()

X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
X16    0
X17    0
X18    0
X19    0
X20    0
X21    0
X22    0
X23    0
X24    0
X25    0
X26    0
X27    0
X28    0
X29    0
X30    0
X31    0
X32    0
X33    0
X34    0
X35    0
X36    0
X37    0
X38    0
X39    0
X40    0
X41    0
X42    0
X43    0
X44    0
X45    0
X46    0
X47    0
X48    0
X49    0
X50    0
X51    0
X52    0
X53    0
X54    0
X55    0
X56    0
X57    0
dtype: int64

In [7]:
# checking the missing values in the output.
y.isnull().sum()

0

## Feature Selection

In [8]:
num_feats = 25

##### 1. Pearson Correlation

In [9]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')


25 selected features


##### 2. Chi-squared

In [10]:
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

25 selected features


##### 3. Recursive Feature Elimination

In [11]:
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 57 features.
Fitting estimator with 47 features.
Fitting estimator with 37 features.
Fitting estimator with 27 features.
25 selected features


##### 4. Lasso

In [12]:
embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

22 selected features


##### 5. Tree-based: SelectFromModel

In [13]:
embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(X, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

14 selected features


In [14]:
feature_name = X.columns.tolist()
# put all selection together
feature_selection_df = pd.DataFrame({'Feature': feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support, 'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,X7,True,True,True,True,True,5
2,X57,True,True,True,True,True,5
3,X53,True,True,True,True,True,5
4,X5,True,True,True,True,True,5
5,X27,True,True,True,True,True,5
6,X25,True,True,True,True,True,5
7,X24,True,True,True,True,True,5
8,X23,True,True,True,True,True,5
9,X21,True,True,True,True,True,5
10,X16,True,True,True,True,True,5


In [15]:
type(feature_selection_df['Feature'])

pandas.core.series.Series

In [16]:
reduced_X = pd.DataFrame(X,columns = feature_selection_df['Feature'][:25].tolist())

In [17]:

print(reduced_X)

        X7  X57    X53    X5  X27    X25   X24   X23   X21   X16  ...  X20  \
0     0.00   12  0.000  0.00  0.0   0.00  0.00  0.00  0.00  0.00  ...  0.0   
1     2.25  375  0.000  1.12  0.0   0.00  0.00  0.00  1.12  0.00  ...  0.0   
2     0.00    5  0.000  0.00  0.0   0.00  0.00  0.00  0.00  0.00  ...  0.0   
3     0.00  122  0.370  1.93  0.0   0.00  0.00  0.64  2.58  1.29  ...  0.0   
4     0.58  207  0.239  0.58  0.0   0.00  1.16  0.00  1.74  0.58  ...  0.0   
...    ...  ...    ...   ...  ...    ...   ...   ...   ...   ...  ...  ...   
3905  0.00   12  0.000  0.00  0.0   0.00  0.00  0.00  0.00  0.00  ...  0.0   
3906  0.00    4  0.000  0.00  0.0  18.18  0.00  0.00  0.00  0.00  ...  0.0   
3907  0.00  131  0.000  0.00  0.0   0.00  0.00  0.00  0.51  0.12  ...  0.0   
3908  0.00   81  0.198  0.00  0.0   0.00  0.00  0.00  0.00  0.00  ...  0.0   
3909  0.00  717  0.467  0.00  0.0   0.00  0.48  0.00  0.96  0.48  ...  0.0   

       X19   X18   X17  X37  X30  X56   X46   X45  X42  
0     

In [18]:
# split the dataset in the ration 4:1
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3519, 57) (391, 57) (3519,) (391,)


In [19]:
#split the reduced dataset in the ratio 4:1
X_reduced_train, X_reduced_val, y_reduced_train, y_reduced_val = train_test_split(reduced_X , y, test_size=0.1, random_state=0)
print(X_reduced_train.shape, X_reduced_val.shape, y_reduced_train.shape, y_reduced_val.shape)

(3519, 25) (391, 25) (3519,) (391,)


### Training and testing different algorithms



##### 1. Naive Bayes

In [20]:
mnb = MultinomialNB().fit(X_train, y_train)
print("score on test: " + str(mnb.score(X_val, y_val)))
print("score on train: "+ str(mnb.score(X_train, y_train)))

score on test: 0.7877237851662404
score on train: 0.7945439045183291


In [21]:
reduced_mnb = MultinomialNB().fit(X_reduced_train, y_reduced_train)
print("score on test: " + str(reduced_mnb.score(X_reduced_val, y_reduced_val)))
print("score on train: "+ str(reduced_mnb.score(X_reduced_train, y_reduced_train)))

score on test: 0.7416879795396419
score on train: 0.7601591361182154


##### 2. Logistic Regression 

In [22]:
lr=LogisticRegression(max_iter=2000)
lr.fit(X_train, y_train)
print("score on test: " + str(lr.score(X_val, y_val)))
print("score on train: "+ str(lr.score(X_train, y_train)))

score on test: 0.9360613810741688
score on train: 0.9306621199204319


In [23]:
reduced_lr=LogisticRegression(max_iter=2000)
reduced_lr.fit(X_reduced_train, y_reduced_train)
print("score on test: " + str(reduced_lr.score(X_reduced_val, y_reduced_val)))
print("score on train: "+ str(reduced_lr.score(X_reduced_train, y_reduced_train)))

score on test: 0.9360613810741688
score on train: 0.9195794259732879


##### 3. Decision Tree (Overfitting)

In [24]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print("score on test: "  + str(clf.score(X_val, y_val)))
print("score on train: " + str(clf.score(X_train, y_train)))

score on test: 0.887468030690537
score on train: 0.9997158283603297


In [25]:
reduced_clf = DecisionTreeClassifier()
reduced_clf.fit(X_reduced_train, y_reduced_train)
print("score on test: "  + str(reduced_clf.score(X_reduced_val, y_reduced_val)))
print("score on train: " + str(reduced_clf.score(X_reduced_train, y_reduced_train)))

score on test: 0.8900255754475703
score on train: 0.9994316567206593


#### 4. Neural Network

In [26]:
model=models.Sequential()
model.add(layers.Dense(8,kernel_regularizer=regularizers.l2(0.003),activation='relu',input_shape=(57,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8,kernel_regularizer=regularizers.l2(0.003),activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=250,batch_size=512)
print("score on test: " + str(model.evaluate(X_val,y_val)[1]))
print("score on train: "+ str(model.evaluate(X_train,y_train)[1]))


Epoch 1/250
7/7 [==============================] - 1s 3ms/step - loss: 31.4992 - accuracy: 0.5323
Epoch 2/250
7/7 [==============================] - 0s 3ms/step - loss: 28.9241 - accuracy: 0.5297
Epoch 3/250
7/7 [==============================] - 0s 3ms/step - loss: 23.5658 - accuracy: 0.5377
Epoch 4/250
7/7 [==============================] - 0s 3ms/step - loss: 25.3456 - accuracy: 0.5413
Epoch 5/250
7/7 [==============================] - 0s 3ms/step - loss: 24.1050 - accuracy: 0.5439
Epoch 6/250
7/7 [==============================] - 0s 6ms/step - loss: 24.1464 - accuracy: 0.5431
Epoch 7/250
7/7 [==============================] - 0s 3ms/step - loss: 18.3338 - accuracy: 0.5399
Epoch 8/250
7/7 [==============================] - 0s 4ms/step - loss: 18.5613 - accuracy: 0.5536
Epoch 9/250
7/7 [==============================] - 0s 4ms/step - loss: 18.1529 - accuracy: 0.5499
Epoch 10/250
7/7 [==============================] - 0s 4ms/step - loss: 17.8843 - accuracy: 0.5368
Epoch 11/250
7/7 [=

In [27]:
reduced_model=models.Sequential()
reduced_model.add(layers.Dense(8,kernel_regularizer=regularizers.l2(0.003),activation='relu',input_shape=(25,)))
reduced_model.add(layers.Dropout(0.5))
reduced_model.add(layers.Dense(8,kernel_regularizer=regularizers.l2(0.003),activation='relu'))
reduced_model.add(layers.Dropout(0.6))
reduced_model.add(layers.Dense(1,activation='sigmoid'))
reduced_model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
reduced_model.fit(X_reduced_train,y_reduced_train,epochs=250,batch_size=512)
print("score on test: " + str(reduced_model.evaluate(X_reduced_val,y_reduced_val)[1]))
print("score on train: "+ str(reduced_model.evaluate(X_reduced_train,y_reduced_train)[1]))


Epoch 1/250
7/7 [==============================] - 1s 3ms/step - loss: 17.2034 - accuracy: 0.5220
Epoch 2/250
7/7 [==============================] - 0s 3ms/step - loss: 16.0016 - accuracy: 0.5365
Epoch 3/250
7/7 [==============================] - 0s 3ms/step - loss: 13.6512 - accuracy: 0.5470
Epoch 4/250
7/7 [==============================] - 0s 4ms/step - loss: 16.8327 - accuracy: 0.5442
Epoch 5/250
7/7 [==============================] - 0s 4ms/step - loss: 11.8475 - accuracy: 0.5561
Epoch 6/250
7/7 [==============================] - 0s 4ms/step - loss: 12.8970 - accuracy: 0.5459
Epoch 7/250
7/7 [==============================] - 0s 4ms/step - loss: 10.5208 - accuracy: 0.5578
Epoch 8/250
7/7 [==============================] - 0s 4ms/step - loss: 9.9519 - accuracy: 0.5578
Epoch 9/250
7/7 [==============================] - 0s 4ms/step - loss: 8.8580 - accuracy: 0.5544
Epoch 10/250
7/7 [==============================] - 0s 3ms/step - loss: 8.1634 - accuracy: 0.5772
Epoch 11/250
7/7 [====

## Final model: Logistic Regression model trained on the reduced dataset 

In [30]:
reduced_test_data = pd.DataFrame(test_data ,columns = feature_selection_df['Feature'][:25].tolist())
test_pred = reduced_lr.predict(reduced_test_data)

numpy.ndarray

In [33]:
# save the test dataset predictions in a csv file
pd.DataFrame(test_pred).to_csv('prediction.csv')